In [ ]:
from tests.helper import assertIdenticalList

In [ ]:
import sys
py2 = sys.version_info[0] == 2

In [ ]:
if not py2:
    import pickle
else:
    import cPickle as pickle

In [ ]:
import numpy as np
from numpy import pi
import scipy
import scipy.signal

In [ ]:
from tail.timestream.boundary_distance import boundary_distance

In [ ]:
with open('tests/timestream/prepare_polmask_input.pkl', 'rb') as f:
    if not py2:
        temp = pickle.load(f, encoding='latin1')
    else:
        temp = pickle.load(f)
    weight = temp["weight"]
    width = temp["width"]

In [ ]:
with open('tests/timestream/prepare_polmask_output.pkl', 'rb') as f:
    if not py2:
        temp = pickle.load(f, encoding='latin1')
    else:
        temp = pickle.load(f)
    newpolmask = temp['newpolmask']

In [ ]:
def neighbor_count(mask):
    kernel = [[1, 1, 1], [1, 0, 1], [1, 1, 1]]
    nec = np.int_(np.round(scipy.signal.convolve2d(mask, kernel, mode='same')))
    return nec

In [ ]:
def neighbor_count_cut(mask):
    '''
    Eliminate the single pixel tendrils emanating out of masks
    Require a pixel to have some number of unmasked neighbors
    '''

    kernel = [[1, 1, 1], [1, 0, 1], [1, 1, 1]]
    maska = mask * 1.0

    nmask = np.sum(mask)
    niter = 0
    while True:
        nec = neighbor_count(mask)
        mask *= (nec >= 4)
        nmask2 = np.sum(mask)
        niter += 1
        if nmask2 == nmask:
            break
        nmask = nmask2

In [ ]:
def get_edge_pixels(win):
    '''Gets edge pixels of a 2d image. Edges are defined as a non-zero pixel where any of its 8 neighbors is zero

    @type win: numpy.ndarray
    @param win: the window function
    @rtype: numpy.ndarray
    @return: array with a value of 1 at the edges of the window function
    '''
    # create mask (1 to where it was non-zero)
    mask = np.ones(win.shape)
    mask[win == 0] = 0

    # 8 pixels that are next to each pixel
    offp1x = np.roll(mask, 1, axis=0)
    offm1x = np.roll(mask, -1, axis=0)
    offp1y = np.roll(mask, 1, axis=1)
    offm1y = np.roll(mask, -1, axis=1)
    offp1xp1y = np.roll(offp1x, 1, axis=1)
    offp1xm1y = np.roll(offp1x, -1, axis=1)
    offm1xp1y = np.roll(offm1x, 1, axis=1)
    offm1xm1y = np.roll(offm1x, -1, axis=1)

    # Pixels that are not next to the edges are 1 in all shifts so anding everything will return True for anything that
    # is not next to an edge and in the mask
    pix_non_edge = (offp1x > 0) & (offm1x > 0) & (offp1y > 0) & (offm1y > 0) & (
        offp1xp1y > 0) & (offp1xm1y > 0) & (offm1xp1y > 0) & (offm1xm1y > 0) & (mask > 0)
    edge_pix = pix_non_edge != mask

    return edge_pix

In [ ]:
def smooth_window(win, ker_size=16):
    '''Smooth input window so that it and its gradients vanish on the boundary region. This 
    is required for B_pure to be identically 0

    @type win: numpy.ndarray
    @param win: the window function
    @rtype: numpy.ndarray
    @return: the window function smoothed so that its value and its derivative go to 0 at the boundary'''
    # This is my attempt to do some sort of smoothing. Smoothing makes the gradient zero at the edges (hopefully), while
    # keeping the mask zero at the edges of our data
    win_temp = 1.0 * win

    for i in range(ker_size // 2):
        win_temp[get_edge_pixels(win_temp)] = 0

    #ker = np.ones([ker_size,ker_size])

    k1 = np.hamming(ker_size)
    k2x, k2y = np.meshgrid(k1, k1)
    ker = k2x * k2y
    ker /= np.sum(ker)

    win_smooth = scipy.signal.fftconvolve(win_temp, ker, mode='same')
    return win_smooth

In [ ]:
def clamp_edge(in_mask, nexm, windowtype='hamming'):
    n = in_mask.shape[0]
    extra_mask = np.ones(n)
    #edgecover = np.hamming(nexm*2)[:nexm]
    edgecover = scipy.signal.get_window(windowtype, nexm * 2)[:nexm]
    extra_mask[:nexm] *= edgecover
    extra_mask[-nexm:] *= edgecover[::-1]
    extra_mask1, extra_mask2 = np.meshgrid(extra_mask, extra_mask)
    extra_mask = extra_mask1 * extra_mask2
    mask = in_mask * extra_mask
    return mask

In [ ]:
def prepare_polmask(weight, width):
    '''
    mask is boolean pixels to use that will specify the apodization
    weight is the noise weighting
    '''
    mask = weight / \
        np.max(weight) if np.max(weight) > 0. else np.zeros_like(weight)
    mask = clamp_edge(mask, 64)
    goodpix = mask > 0.01
    weight = smooth_window(weight)

    neighbor_count_cut(goodpix)

    distance = boundary_distance(goodpix)

    c2mask = np.zeros_like(distance)
    c2mask = (1.0 / (2.0 * pi)) * np.sin(-2.0 * pi *
                                         (distance / width)) + distance / width
    c2mask[distance >= width] = 1.0

    mask = weight / \
        np.max(weight) if np.max(weight) > 0. else np.zeros_like(weight)
    newpolmask = mask * c2mask

    return newpolmask

In [ ]:
newpolmask2 = prepare_polmask(weight,width)

In [ ]:
assertIdenticalList(newpolmask, newpolmask2, atol=0.03)

# Investigate

In [ ]:
print(weight.shape, newpolmask.shape)

In [ ]:
print(weight.dtype, type(width), newpolmask.dtype)